In [ ]:

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

system_message = """You are an expert in solving technical problems, give short and concise answers, and do not exceed two sentenses, make the answer simple and easy to understand"""


def getInfoAboutPersonalTechnicalChallenges(user_input):
    #simulate datbase call 
    print(user_input)
    return "I have a problem with my computer, it is slow and I think it is because of the virus"


def getInfoAboutColleaguesTechnicalChallenges(user_input):
    #simulate datbase call 
    print(user_input)
    return "James has issues with his computer, he thinks it is because of the virus"

personalChallengesFunction = {
    "name": "getInfoAboutPersonalTechnicalChallenges",
    "description": "Get information about personal technical challenges",
    "parameters": {
        "type": "object",
        "properties": {
            "user_input": {"type": "string", "description": "The user's input"} 
        },
        "required": ["user_input"]
    }   
}

colleagueChallengesFunction = {
    "name": "getInfoAboutColleaguesTechnicalChallenges",
    "description": "Get information about a colleague's technical challenges",
    "parameters": {
        "type": "object",
        "properties": {
            "user_input": {"type": "string", "description": "The user's input"}
        },
        "required": ["user_input"]
    }
}

tools = [
    {"type": "function", "function": personalChallengesFunction},
    {"type": "function", "function": colleagueChallengesFunction}
]


def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        user_input = arguments.get("user_input")

        if tool_call.function.name == "getInfoAboutPersonalTechnicalChallenges":
            result = getInfoAboutPersonalTechnicalChallenges(user_input)
        elif tool_call.function.name == "getInfoAboutColleaguesTechnicalChallenges":
            result = getInfoAboutColleaguesTechnicalChallenges(user_input)

        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    return responses

def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    

    return response.choices[0].message.content


gr.ChatInterface(fn=chat, type="messages", title="Personal Assistant", description="Ask me any tehcnical question, including mine and those of my colleagues").launch()



* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


how is james
how is james
What are the issues I have reported so far?
